### Import required libraries

In [1]:
import requests

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes #uncomment if install needed
import folium 

print('Libraries imported.')

Libraries imported.


### Import the required data from wikipedia page in the form of a list and convert it into a dataframe using Pandas 

In [2]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
toronto_data=pd.read_html(wiki_url)
type(toronto_data)

list

In [4]:
toronto_data=toronto_data[0]
toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
toronto_df=pd.DataFrame(toronto_data)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
toronto_df.shape

(180, 3)

### Remove cells with a borough that is Not assigned

In [7]:
toronto_df=toronto_df[toronto_df.Borough!='Not assigned']

In [8]:
toronto_df=toronto_df.reset_index(drop=True)

In [9]:
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
toronto_df.shape

(103, 3)

### Check if there is any cell with neighbourhood that is Not assigned

In [11]:
if(toronto_df['Neighbourhood'].str.contains('Not assigned').any()):
    print ('True')
else:
    print('False')

False


### Check if a postal code containg more than 1 neighbourood

In [12]:
toronto_df['Postal Code'].value_counts(sort=True, ascending=False)[0:5]

M4B    1
M1R    1
M1P    1
M5R    1
M6J    1
Name: Postal Code, dtype: int64

In [13]:
toronto_df.shape

(103, 3)

### Add latitude and longitude data for each postal code using geocoder library

In [14]:
import geocoder

In [15]:
lat_lng_coords = None
latitude=[]
longitude=[]
for postal_code in toronto_df['Postal Code']:
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    lat_lng_coords=None

**Note:** Sincle geocoder.google always returned none, geocoder.arcgis is used 

In [16]:
toronto_df['Latitude']=latitude
toronto_df['Longitude']=longitude
pd.set_option('display.max_rows', None)
toronto_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66277,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
7,M3B,North York,Don Mills,43.74929,-79.36169
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70794,-79.31160
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818


In [17]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Now let's explore and cluster the neighborhoods in Toronto using Foursquare

#### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'MZIJC3XZIK5NQATIOFACON2FMCDOEF2WXDMZSI24RWAKSYPT' 
CLIENT_SECRET = 'ZGCS44WZIZJEREVEALG5BV0ODXP14DP33ZM0AKE0J3VL0PJ3' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MZIJC3XZIK5NQATIOFACON2FMCDOEF2WXDMZSI24RWAKSYPT
CLIENT_SECRET:ZGCS44WZIZJEREVEALG5BV0ODXP14DP33ZM0AKE0J3VL0PJ3


### Explore Neighborhoods in Toronto


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
        results = requests.get(url,headers=headers).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [22]:
toronto_df.loc[0, 'Neighbourhood']

'Parkwoods'

In [23]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] 

neighborhood_name = toronto_df.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.75188000000003, -79.33035999999998.


In [24]:

LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit= {}'.format(CLIENT_ID,
    CLIENT_SECRET,
    VERSION,                                                                                                                         
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
    

url

'https://api.foursquare.com/v2/venues/explore?&client_id=MZIJC3XZIK5NQATIOFACON2FMCDOEF2WXDMZSI24RWAKSYPT&client_secret=ZGCS44WZIZJEREVEALG5BV0ODXP14DP33ZM0AKE0J3VL0PJ3&v=20180605&ll=43.75188000000003,-79.33035999999998&radius=1000&limit= 100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f296739a12e764cfdff03c7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 43.76088000900004,
    'lng': -79.31792376617756},
   'sw': {'lat': 43.74287999100002, 'lng': -79.34279623382241}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display',

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(2420, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75188,-79.33036,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75188,-79.33036,PetSmart,43.748639,-79.333488,Pet Store
2,Parkwoods,43.75188,-79.33036,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.75188,-79.33036,Sun Life,43.754760,-79.332783,Construction & Landscaping
4,Victoria Village,43.73042,-79.31282,Memories of Africa,43.726602,-79.312427,Grocery Store


In [28]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,8,8,8,8,8,8
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,67,67,67,67,67,67
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 261 uniques categories.


### Analyze Each Neighborhood

In [30]:
n_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

n_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [n_onehot.columns[-1]] + list(n_onehot.columns[:-1])
n_onehot = n_onehot[fixed_columns]

n_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [31]:
n_onehot.shape


(2420, 261)

In [32]:
n_grouped = n_onehot.groupby('Neighborhood').mean().reset_index()
n_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0

In [33]:
n_grouped.shape


(98, 261)

In [34]:
num_top_venues = 5

for hood in n_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = n_grouped[n_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0              Pool  0.12
1  Sushi Restaurant  0.12
2   Badminton Court  0.12
3              Park  0.12
4      Skating Rink  0.12


----Alderwood, Long Branch----
               venue  freq
0                Gym   0.2
1  Convenience Store   0.2
2        Pizza Place   0.2
3     Sandwich Place   0.2
4                Pub   0.2


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Park   1.0
1                Yoga Studio   0.0
2                 Nail Salon   0.0
3  Middle Eastern Restaurant   0.0
4         Miscellaneous Shop   0.0


----Bayview Village----
                        venue  freq
0                       Trail  0.25
1                        Park  0.25
2                     Dog Run  0.25
3  Construction & Landscaping  0.25
4          Miscellaneous Shop  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1    Sushi Restaurant

                        venue  freq
0                 Coffee Shop  0.50
1                   Nightclub  0.25
2                        Park  0.25
3           Mobile Phone Shop  0.00
4  Modern European Restaurant  0.00


----Humewood-Cedarvale----
           venue  freq
0   Hockey Arena  0.17
1          Field  0.17
2           Park  0.17
3          Trail  0.17
4  Grocery Store  0.17


----India Bazaar, The Beaches West----
                  venue  freq
0             Pet Store  0.05
1                   Gym  0.05
2     Food & Drink Shop  0.05
3     Fish & Chips Shop  0.05
4  Fast Food Restaurant  0.05


----Islington Avenue, Humber Valley Village----
           venue  freq
0       Pharmacy  0.22
1  Grocery Store  0.11
2           Park  0.11
3           Café  0.11
4  Shopping Mall  0.11


----Kennedy Park, Ionview, East Birchmount Park----
                venue  freq
0   Convenience Store  0.17
1      Discount Store  0.17
2         Coffee Shop  0.17
3  Chinese Restaurant  0.08
4            B

                 venue  freq
0                Diner  0.08
1   Italian Restaurant  0.06
2              Brewery  0.06
3          Coffee Shop  0.06
4  Arts & Crafts Store  0.04


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                venue  freq
0         Coffee Shop  0.29
1  Light Rail Station  0.29
2                Café  0.14
3         Supermarket  0.14
4        Liquor Store  0.14


----The Annex, North Midtown, Yorkville----
                           venue  freq
0             Italian Restaurant  0.10
1                           Café  0.07
2             Mexican Restaurant  0.07
3                    Coffee Shop  0.07
4  Vegetarian / Vegan Restaurant  0.07


----The Beaches----
                 venue  freq
0                Trail  0.25
1    Health Food Store  0.25
2                  Pub  0.25
3  Moroccan Restaurant  0.00
4          Music Store  0.00


----The Danforth West, Riverdale----
            venue  freq
0        Bus Line  0.17
1  Ice Cream Shop  

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = n_grouped['Neighborhood']

for ind in np.arange(n_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(n_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Park,Discount Store,Badminton Court,Supermarket,Pool,Sushi Restaurant,Shopping Mall,Skating Rink,Falafel Restaurant,Event Space
1,"Alderwood, Long Branch",Pizza Place,Pub,Gym,Convenience Store,Sandwich Place,Women's Store,Eastern European Restaurant,Distribution Center,Dog Run,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,Bayview Village,Dog Run,Construction & Landscaping,Park,Trail,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Sushi Restaurant,Pharmacy,Pizza Place,Café,Pub,Butcher,Restaurant


### Using K-means to cluster neighbourhoods

In [37]:
kclusters = 5

n_clustered = n_grouped.drop('Neighborhood', 1)

kmeans = KMeans( n_clusters=kclusters, init='k-means++',random_state=0, n_init=10).fit(n_clustered)

kmeans.labels_

array([0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 2, 4, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int32)

In [38]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) 
n_merged = toronto_df
n_merged = n_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', )
n_merged.reset_index
n_merged.sort_values(by='Cluster Labels', ascending=False)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.81781,-79.28024,4.0,Pharmacy,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.74809,-79.44401,3.0,Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65369,-79.51112,3.0,Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.69479,-79.41440,3.0,Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552,2.0,Home Service,Furniture / Home Store,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
62,M5N,Central Toronto,Roselawn,43.71194,-79.41912,2.0,Home Service,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
0,M3A,North York,Parkwoods,43.75188,-79.33036,1.0,Pet Store,Construction & Landscaping,Park,Food & Drink Shop,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
66,M2P,North York,York Mills West,43.74787,-79.40029,1.0,Speakeasy,Park,Convenience Store,Coffee Shop,Women's Store,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.68811,-79.33418,1.0,Intersection,Home Service,Park,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
69,M6P,West Toronto,"High Park, The Junction South",43.65994,-79.46302,1.0,Convenience Store,Park,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant


Let's visualize the Clusters 

In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(n_merged['Latitude'], n_merged['Longitude'], n_merged['Neighbourhood'], n_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's examine the Clusters

#### Cluster 1

In [40]:
n_merged.loc[n_merged['Cluster Labels'] == 0, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Grocery Store,Intersection,Nail Salon,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Discount Store
2,Downtown Toronto,0.0,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Gym / Fitness Center,Italian Restaurant,Food Truck,Event Space,Electronics Store
3,North York,0.0,Clothing Store,Furniture / Home Store,Women's Store,Men's Store,Cosmetics Shop,Food Court,American Restaurant,Bookstore,Toy / Game Store,Restaurant
4,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Park,Café,Pub,Concert Hall,Music Store,Burger Joint,Restaurant,Chinese Restaurant
5,Etobicoke,0.0,Pharmacy,Shopping Mall,Café,Grocery Store,Bank,Park,Home Service,Skating Rink,Farm,Falafel Restaurant
7,North York,0.0,Coffee Shop,Gas Station,Beer Store,Supermarket,Park,Gym,Burger Joint,Intersection,Donut Shop,Dumpling Restaurant
8,East York,0.0,Pizza Place,Pharmacy,Athletics & Sports,Breakfast Spot,Café,Rock Climbing Spot,Bank,Fast Food Restaurant,Gastropub,Intersection
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Hotel,Café,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Furniture / Home Store,Diner,Middle Eastern Restaurant
10,North York,0.0,Pizza Place,Mediterranean Restaurant,Grocery Store,Sushi Restaurant,Japanese Restaurant,Latin American Restaurant,Ice Cream Shop,Asian Restaurant,Fast Food Restaurant,Gas Station
11,Etobicoke,0.0,Jewelry Store,Movie Theater,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


#### Cluster 2

In [41]:
n_merged.loc[n_merged['Cluster Labels'] == 1, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Pet Store,Construction & Landscaping,Park,Food & Drink Shop,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
18,Scarborough,1.0,Construction & Landscaping,Park,Gym / Fitness Center,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
22,Scarborough,1.0,Business Service,Construction & Landscaping,Park,Coffee Shop,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
35,East York,1.0,Intersection,Home Service,Park,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
39,North York,1.0,Dog Run,Construction & Landscaping,Park,Trail,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
49,North York,1.0,Bakery,Park,Basketball Court,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store
61,Central Toronto,1.0,Photography Studio,Park,Electronics Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
66,North York,1.0,Speakeasy,Park,Convenience Store,Coffee Shop,Women's Store,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
69,West Toronto,1.0,Convenience Store,Park,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant
73,Central Toronto,1.0,Garden,Park,Gym Pool,Playground,Electronics Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


#### Cluster 3

In [42]:
n_merged.loc[n_merged['Cluster Labels'] == 2, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2.0,Home Service,Furniture / Home Store,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
62,Central Toronto,2.0,Home Service,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 4

In [43]:
n_merged.loc[n_merged['Cluster Labels'] == 3, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,3.0,Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
68,Central Toronto,3.0,Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
98,Etobicoke,3.0,Park,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 5

In [44]:
n_merged.loc[n_merged['Cluster Labels'] == 4, n_merged.columns[[1] + list(range(5, n_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Scarborough,4.0,Pharmacy,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
